<a href="https://colab.research.google.com/github/kavyaaa57/AIES_SENTIMENTAL_ANALYSIS/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,SimpleRNN
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AIES/notes_workbook/Tweets.csv')
data = data[['text','airline_sentiment']]

In [ ]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
data = data[data['airline_sentiment'] != "neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-zA-z0-9\s]', '', x))
data['text'] = data['text'].apply(lambda x: x.replace('rt', ''))
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
print(data[data['airline_sentiment'] == 'positive'].size)
print(data[data['airline_sentiment'] == 'negative'].size)
print(X)


<ipython-input-4-0b448f4f21f5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-4-0b448f4f21f5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-zA-z0-9\s]', '', x))
<ipython-input-4-0b448f4f21f5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

4726
18356
[[   0    0    0 ...    1    2  170]
 [   0    0    0 ...   53   25  469]
 [   0    0    0 ...  473   85   21]
 ...
 [   0    0    0 ...    8    1  479]
 [   0    0    0 ... 1618   44  911]
 [   0    0    0 ...   46  138   11]]


In [ ]:
embed_dim = 128
units = 196

In [ ]:
rnn_model = Sequential()
rnn_model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
rnn_model.add(SpatialDropout1D(0.4))
rnn_model.add(SimpleRNN(units, dropout=0.2, recurrent_dropout=0.2))
rnn_model.add(Dense(2, activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 32, 128)           0         
 Dropout1D)                                                      
                                                                 
 simple_rnn (SimpleRNN)      (None, 196)               63700     
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 320094 (1.22 MB)
Trainable params: 320094 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 32, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spati  (None, 32, 128)           0         
 alDropout1D)                                                    
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
Y = pd.get_dummies(data['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7732, 32) (7732, 2)
(3809, 32) (3809, 2)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
242/242 - 44s - loss: 0.3334 - accuracy: 0.8646 - 44s/epoch - 182ms/step
Epoch 2/7
242/242 - 41s - loss: 0.1784 - accuracy: 0.9296 - 41s/epoch - 168ms/step
Epoch 3/7
242/242 - 40s - loss: 0.1396 - accuracy: 0.9483 - 40s/epoch - 164ms/step
Epoch 4/7
242/242 - 39s - loss: 0.1145 - accuracy: 0.9580 - 39s/epoch - 159ms/step
Epoch 5/7
242/242 - 39s - loss: 0.0955 - accuracy: 0.9640 - 39s/epoch - 160ms/step
Epoch 6/7
242/242 - 40s - loss: 0.0784 - accuracy: 0.9670 - 40s/epoch - 164ms/step
Epoch 7/7
242/242 - 40s - loss: 0.0673 - accuracy: 0.9741 - 40s/epoch - 163ms/step


In [ ]:
rnn_model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
242/242 - 8s - loss: 0.5137 - accuracy: 0.7892 - 8s/epoch - 33ms/step
Epoch 2/7
242/242 - 8s - loss: 0.3495 - accuracy: 0.8557 - 8s/epoch - 33ms/step
Epoch 3/7
242/242 - 6s - loss: 0.2563 - accuracy: 0.8983 - 6s/epoch - 26ms/step
Epoch 4/7
242/242 - 8s - loss: 0.1884 - accuracy: 0.9250 - 8s/epoch - 33ms/step
Epoch 5/7
242/242 - 6s - loss: 0.1639 - accuracy: 0.9400 - 6s/epoch - 27ms/step
Epoch 6/7
242/242 - 8s - loss: 0.1303 - accuracy: 0.9525 - 8s/epoch - 33ms/step
Epoch 7/7
242/242 - 6s - loss: 0.1377 - accuracy: 0.9512 - 6s/epoch - 27ms/step


In [ ]:
model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

120/120 - 3s - loss: 0.3200 - accuracy: 0.9118 - 3s/epoch - 23ms/step


[0.3199610114097595, 0.9117878675460815]

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
while True:
    text = input("Enter the text (or type 'quit' to exit): ")

    if text.lower() == 'quit':
        print("Exiting...")
        break

    text_sequence = tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, maxlen=32, dtype='int32', value=0)

    airline_sentiment = model.predict(text_sequence, batch_size=1, verbose=2)[0]
    predicted_class = np.argmax(airline_sentiment)

    if predicted_class == 0:
        print("Negative")
    elif predicted_class == 1:
        print("Positive")


Enter the text (or type 'quit' to exit): hi
1/1 - 0s - 284ms/epoch - 284ms/step
Positive
Enter the text (or type 'quit' to exit): you are spending most of your time in social media
1/1 - 0s - 35ms/epoch - 35ms/step
Negative
Enter the text (or type 'quit' to exit): you like studying but still chose to not study
1/1 - 0s - 32ms/epoch - 32ms/step
Negative
Enter the text (or type 'quit' to exit): the coffee is too hot to drink
1/1 - 0s - 59ms/epoch - 59ms/step
Positive
Enter the text (or type 'quit' to exit): the series is taking too long to execute
1/1 - 0s - 58ms/epoch - 58ms/step
Negative
Enter the text (or type 'quit' to exit): i hate books but now I am used to it
1/1 - 0s - 61ms/epoch - 61ms/step
Negative
Enter the text (or type 'quit' to exit): I hate books but now I adore them
1/1 - 0s - 32ms/epoch - 32ms/step
Negative
Enter the text (or type 'quit' to exit): tamil is old language
1/1 - 0s - 59ms/epoch - 59ms/step
Negative
Enter the text (or type 'quit' to exit): richer are richer a